In [3]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
import numpy as np # library to handle data in a vectorized manner

print('Libraries imported.')

Libraries imported.


In [7]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
from bs4 import BeautifulSoup
bsoup = BeautifulSoup(website_url,'html')
#print(soup.prettify())


## **here we will export the table from wikipedia on postal codes**

In [8]:
My_table = bsoup.find('table',{'class':'wikitable sortable'})
#My_table

In [9]:
table_body = My_table.find('tbody')
rows = table_body.find_all('tr')
l=[]
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    l.append(cols)
df_PC=pd.DataFrame(l, columns=["Postal Code","Borough","Neighborhood"])
df_PC=df_PC.drop(df_PC.index[[0]]).reset_index(drop=True)
#df_PC

## **now, we need to get rid of not assigned in borough**


In [10]:
df_drop = df_PC.drop(df_PC[df_PC.Borough == 'Not assigned'].index).reset_index(drop=True)
df_drop.head(5)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [11]:
for index, row in df_drop.iterrows():
    if row ['Neighborhood']=='Not assigned':
        row ['Neighborhood']= row ['Borough']
df_drop.head(5)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [12]:
df_final = df_drop.groupby(['Postal Code', 'Borough'])['Neighborhood'].apply(lambda x: ', '.join(x.astype(str))).reset_index()
df_final.head(5)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


## **now we need to look at geographical coordinates**

In [13]:
DF_LL = pd.read_csv("https://cocl.us/Geospatial_data")
DF_LL.columns = ['Postal Code', 'Latitude', 'Longitude']
DF_LL.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
DF_merged=pd.merge(DF_LL,df_final , on='Postal Code')
DF_merged.head(5)

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M1B,43.806686,-79.194353,Scarborough,"Rouge, Malvern"
1,M1C,43.784535,-79.160497,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae


## **first create map of Toronto then focus on one zone**

In [2]:
map_toronto = folium.Map(location=[43.6532, -79.3832], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(DF_merged['Latitude'], DF_merged['Longitude'], DF_merged['Borough'], DF_merged['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

NameError: name 'folium' is not defined

## **we want to look just at downtown Toronto for UofT**

In [20]:
DT_data = DF_merged[DF_merged['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
DT_data

,Postal Code,Latitude,Longitude,Borough,Neighborhood
0,M4W,43.679563,-79.377529,Downtown Toronto,Rosedale
1,M4X,43.667967,-79.367675,Downtown Toronto,"Cabbagetown, St. James Town"
2,M4Y,43.665860,-79.383160,Downtown Toronto,Church and Wellesley
3,M5A,43.654260,-79.360636,Downtown Toronto,"Harbourfront, Regent Park"
4,M5B,43.657162,-79.378937,Downtown Toronto,"Ryerson, Garden District"
5,M5C,43.651494,-79.375418,Downtown Toronto,St. James Town
6,M5E,43.644771,-79.373306,Downtown Toronto,Berczy Park
7,M5G,43.657952,-79.387383,Downtown Toronto,Central Bay Street
8,M5H,43.650571,-79.384568,Downtown Toronto,"Adelaide, King, Richmond"
9,M5J,43.640816,-79.381752,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station"


## **we need to get the geographical coordinates of DT Toronto**

In [17]:
latitude=43.6543
longitude=-79.3860

## **DT Toronto map**

In [18]:
map_DT = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(DT_data['Latitude'], DT_data['Longitude'], DT_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_DT)  
    
map_DT

## **connect with foursquare**

In [19]:
CLIENT_ID = '22LA0W1UYAA2MYXY0WMLO3ZR1Y0Y2D5TGXELMPXQEOC1OF4S' # your Foursquare ID
CLIENT_SECRET = '1ZLFGQJ0EJHIGNRTYC4TVVSB3LT0ACLLAQJ32KTN5LUQ1MAP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 22LA0W1UYAA2MYXY0WMLO3ZR1Y0Y2D5TGXELMPXQEOC1OF4S
CLIENT_SECRET:1ZLFGQJ0EJHIGNRTYC4TVVSB3LT0ACLLAQJ32KTN5LUQ1MAP


## **note that loc=12 is UofT, so let's see that is close by that place**

In [21]:
DT_data.loc[12, 'Neighborhood']

'Harbord, University of Toronto'

In [22]:
#get the lat and long
neighborhood_latitude = DT_data.loc[12, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = DT_data.loc[12, 'Longitude'] # neighborhood longitude value

neighborhood_name = DT_data.loc[12, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Harbord, University of Toronto are 43.6626956, -79.4000493.


## **now let's see the first 100 venues in 1 km of range**

In [23]:
# type your answer here
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=22LA0W1UYAA2MYXY0WMLO3ZR1Y0Y2D5TGXELMPXQEOC1OF4S&client_secret=1ZLFGQJ0EJHIGNRTYC4TVVSB3LT0ACLLAQJ32KTN5LUQ1MAP&v=20180605&ll=43.6626956,-79.4000493&radius=1000&limit=100'

In [25]:
results = requests.get(url).json()

In [26]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [27]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Yasu,Japanese Restaurant,43.662837,-79.403217
1,Rasa,Restaurant,43.662757,-79.403988
2,Her Father's Cider Bar + Kitchen,Beer Bar,43.662448,-79.404703
3,Piano Piano,Italian Restaurant,43.662949,-79.402898
4,The Dessert Kitchen,Dessert Shop,43.662823,-79.402746


## **we observe how the first venues are all restaurant**

## **then we can look at different areas and see if there is one with different categories**

In [28]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [29]:
DT_venues = getNearbyVenues(names=DT_data['Neighborhood'],
                                   latitudes=DT_data['Latitude'],
                                   longitudes=DT_data['Longitude']
                                  )

Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie


In [30]:
print(DT_venues.shape)
DT_venues.head()

(1581, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Summerhill Market,43.686265,-79.375458,Grocery Store
1,Rosedale,43.679563,-79.377529,Toronto Lawn Tennis Club,43.680667,-79.388559,Athletics & Sports
2,Rosedale,43.679563,-79.377529,Black Camel,43.677016,-79.389367,BBQ Joint
3,Rosedale,43.679563,-79.377529,Craigleigh Gardens,43.678099,-79.371586,Park
4,Rosedale,43.679563,-79.377529,Pie Squared,43.672143,-79.377856,Pie Shop


## **for each neighborhood in downtown we have:**

In [31]:
DT_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,100,100,100,100,100,100
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",39,39,39,39,39,39
Central Bay Street,100,100,100,100,100,100
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100


In [32]:
DT_onehot = pd.get_dummies(DT_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
DT_onehot['Neighborhood'] = DT_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [DT_onehot.columns[-1]] + list(DT_onehot.columns[:-1])
DT_onehot = DT_onehot[fixed_columns]

DT_onehot.head()

,Yoga Studio,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Rosedale,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

## **to better understand what is the frequency of each venue in the sub neighborhood, we analyze the mean of the frequency of occurrence of each category**

In [44]:
DT_grouped = DT_onehot.groupby('Neighborhood').mean().reset_index()
DT_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,Airport,Airport Lounge,American Restaurant,Animal Shelter,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Bookstore,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Gym,College Rec Center,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Gaming Cafe,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pie Shop,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Restaurant,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,South American Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Art,Supermarket,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tech Startup,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trail,Train Station,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.01,0.00,0.000000,0.000000,0.030000,0.00,0.00,0.00,0.01,0.01,0.02,0.000000,0.00,0.000000,0.00,0.010000,0.000000,0.02,0.00,0.00,0.00,0.02,0.000000,0.00,0.000000,0.01,0.01,0.010000,0.00,0.00,0.00,0.000000,0.01,0.01,0.000000,0.050000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.03,0.00,0.050000,0.00,0.00,0.00,0.00,0.00,0.02,0.000000,0.02,0.00,0.00,0.000000,0.020000,0.01,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00,0.00,0.01,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.01,0.00,0.00,0.00,0.00,0.02,0.00,0.000000,0.020000,0.00,0.00,0.00,0.000000,0.00,0.01,0.000000,0.02,0.01,0.00,0.000000,0.00,0.00,0.00,0.00,0.00,0.000000,0.05,0.00,0.00,0.000000,0.00,0.010000,0.020000,0.01,0.000000,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.01,0.00,0.01,0.00,0.000000,0.00,0.00,0.00,0.00,0.01,0.02,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.01,0.00,0.00,0.00,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.02,0.000000,0.01,0.01,0.000000,0.00,0.010000,0.01,0.01,0.020000,0.000000,0.00,0.00,0.00,0.01,0.000000,0.000000,0.00,0.000000,0.01,0.00,0.01,0.00,0.01,0.000000

## **in particular we can look at the neighborhood Harbord, University of Toronto**


In [45]:
DT_grouped.loc[11]

Neighborhood                     Harbord, University of Toronto
Yoga Studio                                                   0
Afghan Restaurant                                             0
Airport                                                       0
Airport Lounge                                                0
American Restaurant                                           0
Animal Shelter                                                0
Aquarium                                                      0
Art Gallery                                                   0
Art Museum                                                    0
Arts & Crafts Store                                        0.03
Asian Restaurant                                              0
Athletics & Sports                                            0
Auto Dealership                                               0
BBQ Joint                                                     0
Bagel Shop                              

## **we can see that the neighborhood Harbord, University of Toronto has arts crafts stores, bagel shop, bar, beer bar, bookstore, grocery and restaurant in general; however no health care centres**

## **now let's look at  Church and Wellesley**

In [49]:
DT_grouped.loc[7]

Neighborhood                     Church and Wellesley
Yoga Studio                                      0.01
Afghan Restaurant                                0.01
Airport                                             0
Airport Lounge                                      0
American Restaurant                              0.01
Animal Shelter                                      0
Aquarium                                            0
Art Gallery                                         0
Art Museum                                          0
Arts & Crafts Store                              0.01
Asian Restaurant                                    0
Athletics & Sports                                  0
Auto Dealership                                     0
BBQ Joint                                           0
Bagel Shop                                          0
Bakery                                              0
Bank                                                0
Bar                         

## **we can see that it offers healthcares centers as well as all other restaurant and ameneties related to study**

## **let'a analyze the first 15 venues**

In [52]:
num_top_venues = 15

for hood in DT_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = DT_grouped[DT_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                     venue  freq
0              Coffee Shop  0.05
1                    Hotel  0.05
2                     Café  0.05
3                  Theater  0.04
4         Sushi Restaurant  0.03
5      American Restaurant  0.03
6           Clothing Store  0.03
7              Pizza Place  0.02
8                 Beer Bar  0.02
9   Furniture / Home Store  0.02
10               Gastropub  0.02
11              Restaurant  0.02
12                     Bar  0.02
13              Steakhouse  0.02
14           Deli / Bodega  0.02


----Berczy Park----
                  venue  freq
0           Coffee Shop  0.09
1                 Hotel  0.06
2                  Café  0.05
3              Beer Bar  0.04
4   Japanese Restaurant  0.04
5            Restaurant  0.03
6          Cocktail Bar  0.03
7                Bakery  0.03
8                  Park  0.03
9                   Gym  0.02
10   Seafood Restaurant  0.02
11       Breakfast Spot  0.02
12           Steakhouse  0.

In [53]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [68]:
num_top_venues = 15

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = DT_grouped['Neighborhood']

for ind in np.arange(DT_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(DT_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,"Adelaide, King, Richmond",Café,Coffee Shop,Hotel,Theater,Sushi Restaurant,Clothing Store,American Restaurant,Deli / Bodega,Pizza Place,Beer Bar,Movie Theater,Bar,Steakhouse,Concert Hall,Cosmetics Shop
1,Berczy Park,Coffee Shop,Hotel,Café,Japanese Restaurant,Beer Bar,Park,Restaurant,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market,Steakhouse,Breakfast Spot,Gastropub,Gym
2,"CN Tower, Bathurst Quay, Island airport, Harbo...",Coffee Shop,Café,Harbor / Marina,Dog Run,Track,Garden,Sculpture Garden,Park,Scenic Lookout,Dance Studio,Airport Lounge,Airport,Dumpling Restaurant,Event Space,Falafel Restaurant
3,"Cabbagetown, St. James Town",Park,Gastropub,Coffee Shop,Diner,Japanese Restaurant,Café,Pet Store,Bakery,Butcher,Farm,Filipino Restaurant,Rock Club,Restaurant,Bistro,Beer Store
4,Central Bay Street,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Park,Japanese Restaurant,Ramen Restaurant,Italian Restaurant,Bubble Tea Shop,Burger Joint,Burrito Place,Clothing Store,Bar,Plaza,Gastropub,Bookstore
5,"Chinatown, Grange Park, Kensington Market",Vegetarian / Vegan Restaurant,Café,Bar,Bakery,Coffee Shop,Mexican Restaurant,Dessert Shop,Art Gallery,Caribbean Restaurant,Burger Joint,Comfort Food Restaurant,Cocktail Bar,Yoga Studio,Ice Cream Shop,Ramen Restaurant
6,Christie,Korean Restaurant,Café,Coffee Shop,Grocery Store,Cocktail Bar,Indian Restaurant,Ice Cream Shop,Comedy Club,Karaoke Bar,Pizza Place,Ethiopian Restaurant,Park,Bar,Sandwich Place,Vegetarian / Vegan Restaurant
7,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Café,Italian Restaurant,Gay Bar,Mediterranean Restaurant,Diner,Dance Studio,Ramen Restaurant,Thai Restaurant,Gastropub,Men's Store,Park,Burger Joint
8,"Commerce Court, Victoria Hotel",Coffee Shop,Hotel,Café,Japanese Restaurant,Beer Bar,American Restaurant,Steakhouse,Restaurant,Gastropub,Concert Hall,Thai Restaurant,Farmers Market,Plaza,Breakfast Spot,Cocktail Bar
9,"Design Exchange, Toronto Dominion Centre",Hotel,Coffee Shop,Café,Italian Restaurant,Steakhouse,Restaurant,American Restaurant,Concert Hall,Theater,Beer Bar,Japanese Restaurant,Plaza,Lounge,Thai Restaurant,Park


we can see now how all the most common venues are cafes and restaurants, then we can see that the neighborhood we can see now how all the most common venues are cafes and restaurants, then we can see that the neighborhood offers other amenities as bookstore and art stores. However, areas like Rosedale Rosedale Rosedale offers restaturant and grocery and even metro station.  

dataframe that includes the cluster as well as the top 15 venues for each neighborhood.

In [69]:
# set number of clusters
kclusters = 5

DT_grouped_clustering = DT_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(DT_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 1, 0, 0, 3, 4, 0, 0, 0], dtype=int32)

In [72]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Label', kmeans.labels_)

DT_merged = DT_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
DT_merged = DT_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

DT_merged.head() # check the last columns!

,Postal Code,Latitude,Longitude,Borough,Neighborhood,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,M4W,43.679563,-79.377529,Downtown Toronto,Rosedale,2,2,Park,Coffee Shop,Bank,Grocery Store,Pie Shop,Smoothie Shop,Building,Candy Store,Caribbean Restaurant,Metro Station,Breakfast Spot,Falafel Restaurant,Bistro,Sandwich Place,Filipino Restaurant
1,M4X,43.667967,-79.367675,Downtown Toronto,"Cabbagetown, St. James Town",0,0,Park,Gastropub,Coffee Shop,Diner,Japanese Restaurant,Café,Pet Store,Bakery,Butcher,Farm,Filipino Restaurant,Rock Club,Restaurant,Bistro,Beer Store
2,M4Y,43.665860,-79.383160,Downtown Toronto,Church and Wellesley,0,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Café,Italian Restaurant,Gay Bar,Mediterranean Restaurant,Diner,Dance Studio,Ramen Restaurant,Thai Restaurant,Gastropub,Men's Store,Park,Burger Joint
3,M5A,43.654260,-79.360636,Downtown Toronto,"Harbourfront, Regent Park",0,0,Coffee Shop,Café,Theater,Italian Restaurant,Bakery,Park,Diner,Breakfast Spot,Gym / Fitness Center,Restaurant,Performing Arts Venue,Brewery,Indian Restaurant,Pub,Gastropub
4,M5B,43.657162,-79.378937,Downtown Toronto,"Ryerson, Garden District",0,0,Coffee Shop,Clothing Store,Italian Restaurant,Restaurant,Gastropub,Middle Eastern Restaurant,Café,Tea Room,Diner,Cosmetics Shop,Department Store,Bookstore,Theater,Japanese Restaurant,Plaza


In [73]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(DT_merged['Latitude'], DT_merged['Longitude'], DT_merged['Neighborhood'], DT_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

let's now analyze the clusters

In [79]:
DT_merged.loc[DT_merged['Cluster Labels'] == 0, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Latitude,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
1,43.667967,0,0,Park,Gastropub,Coffee Shop,Diner,Japanese Restaurant,Café,Pet Store,Bakery,Butcher,Farm,Filipino Restaurant,Rock Club,Restaurant,Bistro,Beer Store
2,43.665860,0,0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Café,Italian Restaurant,Gay Bar,Mediterranean Restaurant,Diner,Dance Studio,Ramen Restaurant,Thai Restaurant,Gastropub,Men's Store,Park,Burger Joint
3,43.654260,0,0,Coffee Shop,Café,Theater,Italian Restaurant,Bakery,Park,Diner,Breakfast Spot,Gym / Fitness Center,Restaurant,Performing Arts Venue,Brewery,Indian Restaurant,Pub,Gastropub
4,43.657162,0,0,Coffee Shop,Clothing Store,Italian Restaurant,Restaurant,Gastropub,Middle Eastern Restaurant,Café,Tea Room,Diner,Cosmetics Shop,Department Store,Bookstore,Theater,Japanese Restaurant,Plaza
5,43.651494,0,0,Coffee Shop,Café,Restaurant,Hotel,Bakery,Gastropub,Breakfast Spot,Italian Restaurant,Seafood Restaurant,Cosmetics Shop,Diner,Cocktail Bar,Farmers Market,Japanese Restaurant,Gym
6,43.644771,0,0,Coffee Shop,Hotel,Café,Japanese Restaurant,Beer Bar,Park,Restaurant,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market,Steakhouse,Breakfast Spot,Gastropub,Gym
7,43.657952,0,0,Coffee Shop,Café,Vegetarian / Vegan Restaurant,Park,Japanese Restaurant,Ramen Restaurant,Italian Restaurant,Bubble Tea Shop,Burger Joint,Burrito Place,Clothing Store,Bar,Plaza,Gastropub,Bookstore
8,43.650571,0,0,Café,Coffee Shop,Hotel,Theater,Sushi Restaurant,Clothing Store,American Restaurant,Deli / Bodega,Pizza Place,Beer Bar,Movie Theater,Bar,Steakhouse,Concert Hall,Cosmetics Shop
9,43.640816,0,0,Coffee Shop,Hotel,Café,Aquarium,Italian Restaurant,Scenic Lookout,Bakery,Park,Brewery,Gym,Japanese Restaurant,Steakhouse,Fried Chicken Joint,Deli / Bodega,Restaurant
10,43.647177,0,0,Hotel,Coffee Shop,Café,Italian Restaurant,Steakhouse,Restaurant,American Restaurant,Concert Hall,Theater,Beer Bar,Japanese Restaurant,Plaza,Lounge,Thai Restaurant,Park


since it does not present as most common venues anhything related to university, we exclude  cluster 0

In [82]:
DT_merged.loc[DT_merged['Cluster Labels'] == 1, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Latitude,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
14,43.628947,1,1,Coffee Shop,Café,Harbor / Marina,Dog Run,Track,Garden,Sculpture Garden,Park,Scenic Lookout,Dance Studio,Airport Lounge,Airport,Dumpling Restaurant,Event Space,Falafel Restaurant


since it does not present as most common venues anything related to university, we exclude  cluster 1

In [83]:
DT_merged.loc[DT_merged['Cluster Labels'] == 2, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Latitude,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
0,43.679563,2,2,Park,Coffee Shop,Bank,Grocery Store,Pie Shop,Smoothie Shop,Building,Candy Store,Caribbean Restaurant,Metro Station,Breakfast Spot,Falafel Restaurant,Bistro,Sandwich Place,Filipino Restaurant


since it does not pres1nt as most common venues anything related to university, we exclude  cluster 2, even if there is a metro station which might be helpful in deciding where to buy a house

In [84]:
DT_merged.loc[DT_merged['Cluster Labels'] == 3, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Latitude,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
12,43.662696,3,3,Café,Bakery,Bar,Vegetarian / Vegan Restaurant,Restaurant,Mexican Restaurant,Arts & Crafts Store,Bookstore,Burger Joint,Ice Cream Shop,Pub,Dessert Shop,Japanese Restaurant,Beer Bar,Thai Restaurant
13,43.653206,3,3,Vegetarian / Vegan Restaurant,Café,Bar,Bakery,Coffee Shop,Mexican Restaurant,Dessert Shop,Art Gallery,Caribbean Restaurant,Burger Joint,Comfort Food Restaurant,Cocktail Bar,Yoga Studio,Ice Cream Shop,Ramen Restaurant


we can see that there is a location (12=UofT) where we have bookstore and art stores as long as food places. this might be a good place to buy a house for students

In [89]:
DT_merged.loc[DT_merged['Cluster Labels'] == 4, DT_merged.columns[[1] + list(range(5, DT_merged.shape[1]))]]

,Latitude,Cluster Label,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,11th Most Common Venue,12th Most Common Venue,13th Most Common Venue,14th Most Common Venue,15th Most Common Venue
17,43.669542,4,4,Korean Restaurant,Café,Coffee Shop,Grocery Store,Cocktail Bar,Indian Restaurant,Ice Cream Shop,Comedy Club,Karaoke Bar,Pizza Place,Ethiopian Restaurant,Park,Bar,Sandwich Place,Vegetarian / Vegan Restaurant


since it does not present as most common venues anything related to university, we exclude  cluster 4

Hence, we can say that we can get a house in location 12 =Harbord, University of Toronto since there are pubs bars and book stores; or Rosedale with food places and metro station